# Testing Jupyter API from within !
 - https://github.com/jupyter/jupyter/wiki/Jupyter-Notebook-Server-API
 - Retrieving the auth token from redis first
 - Then building some REST call and a quick test with Websocket

In [14]:
# Connect to Redis and retrieve the Jupyter token for auth
import redis

# When using docker compose
redisHost = 'redis-server'
# Connect
r = redis.Redis(host=redisHost, port=6379, db=0)
token = r.get("config:jupyter:token")
print('Token:', token.decode())

Token: be8f68892c19db14


In [15]:
# Call the Jupyter REST API with the token
import requests

# Store a kernel ID for later
this_kernel = ''

# Get the list of kernels
resp = requests.get('http://localhost:8888/api/kernels', data={'token': token})
if resp.status_code != 200:
    # This means something went wrong.
    print('Error code {}'.format(resp.status_code))

first = 1
for k in resp.json():
    print('Kernel', k['name'], k['id'], k['execution_state'])
    if first:
        this_kernel = k['id']
        first = 0

Kernel python3 6ec16df9-fd84-4984-b08a-831d7e1ba7e1 idle
Kernel python3 8f09d0d8-311a-42ed-bb16-81781a9f661b busy
Kernel python3 0d7586b5-6fed-4184-b9c1-33690e434152 idle
Kernel python3 8e966f3e-4e90-4bae-9ab5-c46716921d82 idle


In [16]:
# Get the list of notebooks
resp = requests.get('http://localhost:8888/api/contents', data={'token': token})
if resp.status_code != 200:
    # This means something went wrong.
    print('Error code {}'.format(resp.status_code))
for k in resp.json()['content']:
    if k['type'] == 'notebook':
        print('Name: {} \t size {} \t modified {}'.format(k['name'], k['size'], k['last_modified']))

Name: Untitled.ipynb 	 size 2018 	 modified 2020-12-09T19:21:11.548198Z
Name: Untitled1.ipynb 	 size 6221 	 modified 2020-12-10T22:15:22.896123Z
Name: jupyer-api.ipynb 	 size 4337 	 modified 2020-12-10T22:15:49.926485Z
Name: opencv.ipynb 	 size 96202 	 modified 2020-12-10T21:46:12.904677Z
Name: test1.ipynb 	 size 2657 	 modified 2020-12-08T21:41:28.900538Z
Name: test2.ipynb 	 size 2721 	 modified 2020-12-09T18:06:28.251676Z


In [17]:
# Get info about a kernel
import websockets

kernel = this_kernel # 'd01d751c-e235-4155-be6c-5ea2e77effa4'
uri = 'ws://localhost:8888/api/kernels/{}/channels'.format(kernel)
headers = {'Authorization': 'Token ' + token.decode()}

async with websockets.connect(uri, extra_headers = headers) as ws:
    print('WS connected')
    values = await ws.recv()
    print('WS return', values)

WS connected


CancelledError: 